# A simple tetrode pipeline

Here, we make a simple sorting pipeline for tetrode data, based on data collected by Bri Vandrey in the Nolan Lab (University of Edinburgh). First load in the packages we need and the recording

In [ ]:
from pathlib import Path

import spikeinterface.full as si
import probeinterface as pi

base_data_folder = Path("/project/dlb/Dataset/SpikeInterface")
recording_name = "1544_2023-04-21_09-55-34_of"
path_to_recording = base_data_folder / recording_name

recording = si.read_openephys(path_to_recording, stream_id="CH")

The data comes from four tetrodes. Luckily, `ProbeInterface` has a `generate_tetrode` function. We can use that to make four tetrodes, spaced out a little. We'll combine them all into a `ProbeGroup`

In [ ]:
tetrode_group = pi.ProbeGroup()
for a in range(4):
    one_tetrode = pi.generate_tetrode()
    one_tetrode.move([a*250,0])
    tetrode_group.add_probe(one_tetrode)

# Wire it up
tetrode_group.set_global_device_channel_indices(range(16))

# And set the probe
recording = recording.set_probegroup(tetrode_group)

To make sure we've done the probe properly, let's plot it

In [ ]:
si.plot_probe_map(recording)

Each tetrode is quite far apart, and should (usually) be treated independently. Hence we'll split the recording into groups. Since we attached the Probe as a `ProbeGroup`, it already knows about the groups.

In [ ]:
grouped_recording = recording.split_by('group')


We'll now apply some light preprocessing (note: tetrodes don't do well with common referencing. Read more:  https://spikeinterface.readthedocs.io/en/stable/tutorials/forhowto/plot_1_working_with_tetrodes.html) and check if there are any bad channels

In [ ]:
preprocessed_recording = si.bandpass_filter(grouped_recording)

recording_good_channels = si.detect_and_remove_bad_channels(preprocessed_recording,)

print(recording_good_channels)

Nice. Group 2 only has 3 channels, so a bad channel from that group has been removed.

Motion detection algorithms are for high density probes (not tetrodes). Luckily, tetrodes are usually pretty stable. So we can move on to sorting. The most commonly used algorithm for tetrodes is MountainSort. Let's try the latest version of that. Reading the GitHub page (https://github.com/flatironinstitute/mountainsort5) it sounds like if we don't have drift we should use sorting scheme 2.

In [ ]:
sorting = si.run_sorter(sorter_name="mountainsort5", recording=recording_good_channels, scheme='2')

Since the input recording was a dictionary of the four grouped recording, the sorting output is also a dict of four grouped sortings:

In [ ]:
sorting

Let's now make a sorting analyzer and compute some extensions. PCA metrics are particularly useful for tetrode data. Since the recording is quite small, I'll make an analyzer in memory and then save it later.

In [ ]:
# Note: make sure you use the preprocessed recording here!
analyzer = si.create_sorting_analyzer(sorting, recording_good_channels, format="memory")
analyzer.compute({
    "templates": {},
    "waveforms": {},
    "random_spikes": {},
    "correlograms": {},
    "noise_levels": {'method': 'std'},
    "spike_amplitudes": {},
    "spike_locations": {},
    "template_metrics": {},
    "unit_locations": {},
    "template_similarity": {'method': 'l1'},
    "quality_metrics": {},
    "principal_components": {}
})

Now that the computations are done, I'll save the analyzer

In [ ]:
output_folder = Path("/Users/christopherhalcrow/Work/my_projects/milan")
analyzer.save_as(format="binary_folder", folder = output_folder / "tetrode_analyzer")

Finally, let's export a report to see what our units look like

In [ ]:
si.export_report(analyzer, output_folder = output_folder / "tetrode_report")

Go take a look at the report to see your units. Or use some of the `SpikeInterface` widgets to take a look within the notebook